In [6]:
# 1) SETUP -----------------------------------------------------------------
import sys, os

# Add the parent directory (project root) to Python’s module search path
sys.path.insert(0, os.path.abspath('..'))

import pandas as pd
from src.rfm import compute_rfm, scale_rfm, perform_kmeans, label_high_risk_cluster

df = pd.read_csv("../data/raw/data.csv")
df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'])

snapshot_date = df['TransactionStartTime'].max() + pd.Timedelta(days=1)

rfm = compute_rfm(df, snapshot_date)
rfm_scaled, scaler = scale_rfm(rfm)
clusters, kmeans = perform_kmeans(rfm_scaled)

rfm['Cluster'] = clusters
rfm, profile, high_risk_cluster = label_high_risk_cluster(rfm)

# Merge risk label with features
features = pd.read_csv("../data/processed/processed_features.csv")
features['CustomerId'] = df['CustomerId'].values
merged = features.merge(rfm[['CustomerId', 'is_high_risk']], on='CustomerId', how='left')
merged.to_csv("../data/processed/features_with_labels.csv", index=False)

